# Building Word Embeddings with PyTorch

In [1]:
import numpy as np
import pandas as pd
import torch
import torch.nn  as  nn
import torch.autograd as autograd
import torch.optim as optim
import torch.nn.functional as F  
import re
import itertools
from collections import Counter
import random, math
from numpy.random import multinomial
import nltk
from nltk.corpus import stopwords
from tqdm import trange
from textblob import TextBlob
from nltk.stem.porter import PorterStemmer

#### Check GPU Status

In [2]:
print(torch.cuda.is_available())
print(torch.cuda.device_count())

True
1


## Data
- Trip Advisor hotel reviews
- Sci-fi stories

In [3]:
class DataLoader:
    """Download the files and load them into a dataframe.
    
    Attributes
    ----------
    url: Url of the online dataset
    """
    def __init__(self, url):
        self.url = url

    def load_csv(self):
        url = self.url
#         url = 'https://drive.google.com/uc?id=' + url.split('/')[-2]
        df = pd.read_csv(url)
        return df
    
    def load_txt(self):
        with open(self.url) as f:
            df = f.read()
        return df

Load data from files by inheriting DataLoader.

In [4]:
hotel_url = 'data/hotel.csv'
hotel_loader = DataLoader(hotel_url)
hotel_data = hotel_loader.load_csv()

scifi_url = 'data/scifi.txt'
scifi_loader = DataLoader(scifi_url)
scifi_data = scifi_loader.load_txt()

In [6]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [7]:
# pip install textblob

#### Text Data Preprocessing

In [8]:
class TextCleaner:
    """Clean text data.

    This Cleaner performes text pre-processing through a series of 
    operations while creating a NLP application. The operations include
    Lowercasing, Removing stopwords, Filtering Letters, Correcting Spellings and Stemming.
    """
    def __init__(self):
        self.ps = PorterStemmer()
    
    def clean_hotel_data(self, data):
        corpus = []
        stopwords_english = stopwords.words('english')
        for text in data:
            text = text.lower() # Lowercasing
            text.replace('\n', ' ') 
            text = re.sub('[^a-z ]+', '', text) # Filtering Letters
            textblob_ = TextBlob(text)
            text = textblob_.correct().string   # Correcting Spellings
            new_text = [self.ps.stem(word) for word in text.split()]    # Stemming
            text = ' '.join(new_text)
            corpus.append([w for w in text.split() if w != '' and (self.ps.stem(w) in stopwords_english) == False]) # Removing stopwords
        print("Hotel data preprocessing done.")
        return corpus
    
    def clean_scifi_text(self, text):
        stopwords_english = stopwords.words('english')
        corpus = []
        text = text.lower() # Lowercasing
        text.replace('\n', ' ') 
        text = re.sub('[^a-z ]+', '', text) # Filtering Letters
        textblob_ = TextBlob(text)
        text = textblob_.correct().string   # Correcting Spellings
        new_text = [self.ps.stem(word) for word in text.split()]    # Stemming
        text = ' '.join(new_text)
        corpus.append([w for w in text.split() if w != '' and (self.ps.stem(w) in stopwords_english) == False]) # Removing stopwords
        print("Scifi data preprocessing done.")
        return corpus

Implement Data Preprocessing

In [9]:
cleaner = TextCleaner()
hotel_corpus = cleaner.clean_hotel_data(hotel_data['Review'])
scifi_corpus = cleaner.clean_scifi_text(scifi_data)

Hotel data preprocessing done.
Scifi data preprocessing done.


## Context Generator
- **Subsampling Frequent Words** \\
We balanced the word occurences in the data by calculating the probability of keeping the word i in the corpus. \\
$P(w_i) = \frac{10^{-3}}{p_i}(\sqrt{10^3p_i}+1)$
- **Negative Samples** \\
To accelerate the learning, we chose the sigmoid function as output. Meanwhile, We generated negative examples in case the network does not learn from negative samples. The probability of sampling a negative context word is defined as follows: \\
$P(w_i)=\frac{|w_i|^\frac{3}{4}}{\sum^{n}_{j=1}|w_j|^\frac{3}{4}}$ \\
- **Building BoW** \\
In this model, a text is represented as the bag (multiset) of its words, disregarding grammar and even word order but keeping multiplicity.

In [10]:
class ContextGenerator:
    """Generate the context

    This Generator performs frequent words subsampling and 
    negative samples calculating. It builds a context dataset 
    with negative examples.
    """  
    def __init__(self, corpus, sample_size, context_width):
        self.corpus = corpus
        self.sample_size = sample_size
        self.w = context_width
    
    def subsample_frequent_words(self):
        filtered_corpus = []
        word_counts = dict(Counter(list(itertools.chain.from_iterable(self.corpus))))
        sum_word_counts = sum(list(word_counts.values()))
        word_counts = {word: word_counts[word]/float(sum_word_counts) for word in word_counts}
        for text in self.corpus:
            filtered_corpus.append([])
            for word in text:
                if random.random() < (1+math.sqrt(word_counts[word] * 1e3)) * 1e-3 / float(word_counts[word]):
                    filtered_corpus[-1].append(word)
        return filtered_corpus
    
    def sample_negative(self):
        sample_probability = {}
        word_counts = dict(Counter(list(itertools.chain.from_iterable(self.corpus))))
        normalizing_factor = sum([v**0.75 for v in word_counts.values()])
        for word in word_counts:
            sample_probability[word] = word_counts[word]**0.75 / normalizing_factor
        words = np.array(list(word_counts.keys()))
        while True:
            word_list = []
            sampled_index = np.array(multinomial(self.sample_size, list(sample_probability.values())))
            for index, count in enumerate(sampled_index):
                for _ in range(count):
                     word_list.append(words[index])
            yield word_list
            
    
    def generate_context(self):
        filtered_corpus = self.subsample_frequent_words()
        vocabulary = set(itertools.chain.from_iterable(filtered_corpus))
        context_tuple_list = []
        negative_samples = self.sample_negative()
        index = 0

        for text in filtered_corpus:
            for i, word in enumerate(text):
                first_context_word_index = max(0,i-self.w)
                last_context_word_index = min(i+self.w, len(text))
                for j in range(first_context_word_index, last_context_word_index):
                    if i!=j:
                        context_tuple_list.append((word, text[j], next(negative_samples)))
        print("Generated target and context words pairs.")
        return vocabulary, context_tuple_list

Implement Context Generating on datasets with **both context widths 2 and 5**

In [11]:
context_width_2 = 2
context_width_5 = 5
sample_size = 2

hotel_cg_2 = ContextGenerator(hotel_corpus, sample_size, context_width_2)
hotel_vocab_2, hotel_context_2 = hotel_cg_2.generate_context()

hotel_cg_5 = ContextGenerator(hotel_corpus, sample_size, context_width_5)
hotel_vocab_5, hotel_context_5 = hotel_cg_5.generate_context()

scifi_cg_2 = ContextGenerator(scifi_corpus, sample_size, context_width_2)
scifi_vocab_2, scifi_context_2 = scifi_cg_2.generate_context()

scifi_cg_5 = ContextGenerator(scifi_corpus, sample_size, context_width_5)
scifi_vocab_5, scifi_context_5 = scifi_cg_5.generate_context()

Generated pairs of target and context words
Generated pairs of target and context words
Generated pairs of target and context words
Generated pairs of target and context words


#### Early Stopping

In [12]:
class EarlyStopping:
    """Early Stopping during Learning
    
    This Class aims at stopping learning when the loss does not decrease 
    significantly anymore after a certain number of iterations.
    """
    def __init__(self, patience=5, min_percent_gain=0.1):
        self.patience = patience
        self.loss_list = []
        self.min_percent_gain = min_percent_gain / 100.
        
    def update_loss(self, loss):
        self.loss_list.append(loss)
        if len(self.loss_list) > self.patience:
            del self.loss_list[0]
    
    def stop_training(self):
        if len(self.loss_list) == 1:
            return False
        gain = (max(self.loss_list) - min(self.loss_list)) / max(self.loss_list)
        print("Loss gain: {}%".format(round(100*gain,2)))
        if gain < self.min_percent_gain:
            return True
        else:
            return False

## Learner

In [16]:
import torch
import torch.nn as nn
import torch.autograd as autograd
import torch.optim as optim
import torch.nn.functional as F


class CBOW(nn.Module):

    def __init__(self, embedding_size, vocab_size):
        super(CBOW, self).__init__()
        self.embeddings_target = nn.Embedding(vocab_size, embedding_size)
        self.embeddings_context = nn.Embedding(vocab_size, embedding_size)

    def forward(self, target_word, context_word, negative_example):
        emb_target = self.embeddings_target(target_word)
        emb_context = self.embeddings_context(context_word)
        emb_product = torch.mul(emb_target, emb_context)
        emb_product = torch.sum(emb_product, dim=1)
        out = torch.sum(F.logsigmoid(emb_product))
        emb_negative = self.embeddings_context(negative_example)
        emb_product = torch.bmm(emb_negative, emb_target.unsqueeze(2))
        emb_product = torch.sum(emb_product, dim=1)
        out += torch.sum(F.logsigmoid(-emb_product))
        return -out

The neural network in trained with the following parameters:
- embedding size: 50
- batch size: 5000

In [17]:
class EmbeddingLearner:
    def __init__(self, vocabulary, context, batch_size, epochs):
        self.vocabulary = vocabulary
        self.context_tuple_list = context
        self.batch_size = batch_size
        self.word_to_index = {w: idx for (idx, w) in enumerate(self.vocabulary)}
        self.index_to_word = {idx: w for (idx, w) in enumerate(self.vocabulary)}
        self.vocabulary_size = len(self.vocabulary)
        self.net = CBOW(embedding_size=50, vocab_size=self.vocabulary_size)
        self.epochs = epochs
        
        
    def get_batches(self):
        random.shuffle(self.context_tuple_list)
        batches = []
        batch_target, batch_context, batch_negative = [], [], []
        for i in range(len(self.context_tuple_list)):
            batch_target.append(self.word_to_index[self.context_tuple_list[i][0]])
            batch_context.append(self.word_to_index[self.context_tuple_list[i][1]])
            batch_negative.append([self.word_to_index[w] for w in self.context_tuple_list[i][2]])
            if (i+1) % self.batch_size == 0 or i == len(self.context_tuple_list)-1:
                tensor_target = autograd.Variable(torch.from_numpy(np.array(batch_target)).long())
                tensor_context = autograd.Variable(torch.from_numpy(np.array(batch_context)).long())
                tensor_negative = autograd.Variable(torch.from_numpy(np.array(batch_negative)).long())
                batches.append((tensor_target, tensor_context, tensor_negative))
                batch_target, batch_context, batch_negative = [], [], []
        return batches
        
    def learn(self):

        loss_function = nn.CrossEntropyLoss()
        optimizer = optim.Adam(self.net.parameters())
        early_stopping = EarlyStopping(patience=5, min_percent_gain=0.5)

        for i in trange(0, self.epochs):
            losses = []
            context_tuple_batches = self.get_batches()
            for i in range(len(context_tuple_batches)):
                self.net.zero_grad()
                target_tensor, context_tensor, negative_tensor = context_tuple_batches[i]
                loss = self.net(target_tensor, context_tensor, negative_tensor)
                loss.backward()
                optimizer.step()
                losses.append(loss.data)
            print("Loss: ", np.mean(losses))
            early_stopping.update_loss(np.mean(losses))
            if early_stopping.stop_training():
                break

    
    def get_closest_word(self, word, topn=5):
        word_distance = []
        emb = self.net.embeddings_target
        pdist = nn.PairwiseDistance()
        i = self.word_to_index[word]
        lookup_tensor_i = torch.tensor([i], dtype=torch.long)
        v_i = emb(lookup_tensor_i)
        for j in range(len(self.vocabulary)):
            if j != i:
                lookup_tensor_j = torch.tensor([j], dtype=torch.long)
                v_j = emb(lookup_tensor_j)
                word_distance.append((self.index_to_word[j], float(pdist(v_i, v_j))))
        word_distance.sort(key=lambda x: x[1])
        return word_distance[:topn]

## Training
**CBOW2 for the Hotel Reviews dataset**

In [18]:
hotel_learner_2 = EmbeddingLearner(hotel_vocab_2,hotel_context_2, 5000, 100)
hotel_learner_2.learn()

  1%|          | 1/100 [00:01<02:48,  1.70s/it]

Loss:  34249.586


  2%|▏         | 2/100 [00:02<02:33,  1.56s/it]

Loss:  33389.023
Loss gain: 2.51%


  3%|▎         | 3/100 [00:04<02:21,  1.46s/it]

...

Loss:  5601.584
Loss gain: 4.69%


 99%|█████████▉| 99/100 [02:18<00:01,  1.48s/it]

Loss:  5536.9194
Loss gain: 4.63%


100%|██████████| 100/100 [02:19<00:00,  1.40s/it]

Loss:  5473.533
Loss gain: 4.58%


**CBOW2 for the Sci-Fi story dataset**

In [19]:
scifi_learner_2 = EmbeddingLearner(scifi_vocab_2,scifi_context_2,5000,100)
scifi_learner_2.learn()

  1%|          | 1/100 [00:00<00:58,  1.70it/s]

Loss:  33992.7


  2%|▏         | 2/100 [00:01<00:58,  1.67it/s]

Loss:  33380.402
Loss gain: 1.8%


  3%|▎         | 3/100 [00:01<00:59,  1.64it/s]

...

Loss:  7461.4927
Loss gain: 5.57%


 99%|█████████▉| 99/100 [01:00<00:00,  1.75it/s]

Loss:  7356.306
Loss gain: 5.55%


100%|██████████| 100/100 [01:00<00:00,  1.65it/s]

Loss:  7253.316
Loss gain: 5.52%


**CBOW5 for the Hotel Reviews dataset**

In [20]:
hotel_learner_5 = EmbeddingLearner(hotel_vocab_5,hotel_context_5, 5000, 100)
hotel_learner_5.learn()

  1%|          | 1/100 [00:04<06:58,  4.22s/it]

Loss:  33734.562


  2%|▏         | 2/100 [00:08<06:41,  4.09s/it]

Loss:  32203.855
Loss gain: 4.54%


  3%|▎         | 3/100 [00:11<06:33,  4.06s/it]

...

Loss:  4489.3174
Loss gain: 2.02%


 99%|█████████▉| 99/100 [06:53<00:04,  4.27s/it]

Loss:  4467.1235
Loss gain: 1.99%


100%|██████████| 100/100 [06:57<00:00,  4.18s/it]

Loss:  4445.3765
Loss gain: 1.97%


**CBOW5 for the Sci-Fi story dataset**

In [21]:
scifi_learner_5 = EmbeddingLearner(scifi_vocab_5,scifi_context_5,5000,100)
scifi_learner_5.learn()

  1%|          | 1/100 [00:01<02:53,  1.75s/it]

Loss:  34030.805


  2%|▏         | 2/100 [00:03<03:06,  1.90s/it]

Loss:  33134.617
Loss gain: 2.63%


  3%|▎         | 3/100 [00:05<03:00,  1.86s/it]

...

Loss:  5224.7305
Loss gain: 4.45%


 99%|█████████▉| 99/100 [03:07<00:01,  2.00s/it]

Loss:  5167.8022
Loss gain: 4.39%


100%|██████████| 100/100 [03:09<00:00,  1.89s/it]

Loss:  5112.2095
Loss gain: 4.32%


5 closest words of CBOW2 for the Hotel Reviews dataset

In [22]:
for word in ['bar','window','mouth','pay','teach','jump','tireless','rare','nicest']:
    neighbors = hotel_learner_2.get_closest_word(word)
    print(word, ": ", neighbors, "\n")

bar :  [('place', 3.5057084560394287), ('fine', 3.7182514667510986), ('nice', 3.7379472255706787), ('look', 3.761240005493164), ('day', 3.763062000274658)] 

window :  [('use', 4.510156631469727), ('area', 4.525331974029541), ('hotel', 4.534768581390381), ('right', 4.591760158538818), ('tri', 4.6055779457092285)] 

mouth :  [('andr', 6.032471179962158), ('day', 6.1610260009765625), ('servic', 6.175563335418701), ('beauti', 6.179343223571777), ('pike', 6.217563152313232)] 

pay :  [('locat', 4.316544055938721), ('work', 4.3558735847473145), ('hotel', 4.447457313537598), ('tri', 4.484480857849121), ('close', 4.49414587020874)] 

teach :  [('mind', 7.310356616973877), ('level', 7.9295783042907715), ('food', 7.949304103851318), ('separ', 7.961902141571045), ('grace', 8.074626922607422)] 

jump :  [('downtown', 5.247917175292969), ('place', 5.5001606941223145), ('close', 5.830567836761475), ('center', 5.933122158050537), ('hotel', 5.97814416885376)] 

tireless :  [('experi', 4.8784546852111

5 closest words of CBOW2 for the Sci-Fi story dataset

In [23]:
for word in ['mouth','sport','bedroom','trust','fail','pay','largest','clear','endless']:
    neighbors = scifi_learner_2.get_closest_word(word)
    print(word, ": ", neighbors, "\n")

mouth :  [('brought', 6.275054454803467), ('made', 6.327899932861328), ('codiscover', 6.493941783905029), ('came', 6.604343891143799), ('movi', 6.60573673248291)] 

sport :  [('get', 6.421679973602295), ('kirk', 6.525964260101318), ('ground', 6.579998970031738), ('predatorycreatur', 6.715239524841309), ('long', 6.728081703186035)] 

bedroom :  [('made', 5.800624370574951), ('swam', 5.992005348205566), ('voluptu', 6.217353820800781), ('bombshel', 6.220456123352051), ('judi', 6.3308258056640625)] 

trust :  [('toward', 5.99810266494751), ('kept', 6.206921577453613), ('foreman', 6.311612606048584), ('seiz', 6.458076477050781), ('gaunt', 6.518838882446289)] 

fail :  [('windi', 5.9373369216918945), ('sir', 6.292647838592529), ('want', 6.324713230133057), ('neverth', 6.368040084838867), ('may', 6.474075794219971)] 

pay :  [('list', 6.073314666748047), ('innoc', 6.661491870880127), ('question', 6.699519634246826), ('knowledgeseek', 6.8819780349731445), ('lift', 6.998386859893799)] 

largest

5 closest words of CBOW5 for the Hotel Reviews dataset

In [24]:
for word in ['bar','window','mouth','pay','teach','jump','tireless','rare','nicest']:
    neighbors = hotel_learner_5.get_closest_word(word)
    print(word, ": ", neighbors, "\n")

bar :  [('citi', 1.028738021850586), ('hotel', 1.077404260635376), ('use', 1.1310336589813232), ('quit', 1.1636459827423096), ('thing', 1.1637693643569946)] 

window :  [('got', 1.7456876039505005), ('bathroom', 1.7746179103851318), ('old', 1.7912161350250244), ('larg', 1.8197168111801147), ('clean', 1.8316915035247803)] 

mouth :  [('special', 5.855133056640625), ('money', 5.87908935546875), ('highli', 5.896125793457031), ('amaz', 5.924036026000977), ('terribl', 5.945255279541016)] 

pay :  [('price', 0.9755768179893494), ('friendli', 1.1670482158660889), ('hotel', 1.1920456886291504), ('time', 1.2026610374450684), ('stay', 1.2043038606643677)] 

teach :  [('saw', 6.024971008300781), ('happen', 6.090672492980957), ('upgrad', 6.170861721038818), ('rude', 6.18377685546875), ('inclin', 6.247430801391602)] 

jump :  [('garag', 5.9092535972595215), ('soon', 5.947790145874023), ('doe', 5.955192565917969), ('coffe', 6.004382133483887), ('walk', 6.025463581085205)] 

tireless :  [('took', 3.6

5 closest words of CBOW5 for the Sci-Fi story dataset

In [25]:
for word in ['mouth','sport','bedroom','trust','fail','pay','largest','clear','endless']:
    neighbors = scifi_learner_5.get_closest_word(word)
    print(word, ": ", neighbors, "\n")

mouth :  [('say', 3.7974326610565186), ('earth', 4.0416035652160645), ('alma', 4.0852508544921875), ('toward', 4.085824012756348), ('one', 4.2075958251953125)] 

sport :  [('depress', 6.369104862213135), ('good', 6.414592266082764), ('homicid', 6.4989190101623535), ('roar', 6.499039649963379), ('hit', 6.516376495361328)] 

bedroom :  [('peopl', 4.857330322265625), ('got', 5.0196075439453125), ('wa', 5.117725372314453), ('thi', 5.138413429260254), ('came', 5.192033767700195)] 

trust :  [('poor', 7.949508190155029), ('go', 7.997661113739014), ('come', 8.091782569885254), ('cours', 8.12215805053711), ('lift', 8.211256980895996)] 

fail :  [('right', 4.795498847961426), ('light', 4.795810699462891), ('hi', 4.797901153564453), ('day', 4.801190376281738), ('came', 4.8309173583984375)] 

pay :  [('said', 5.613862037658691), ('interest', 6.000959873199463), ('fourth', 6.032034873962402), ('chair', 6.047975540161133), ('say', 6.069349765777588)] 

largest :  [('man', 4.712806701660156), ('say'